# Acquire

In [1]:
# General Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# AWS
import logging
import boto3
from botocore.exceptions import ClientError

# NLP Libraries
import unicodedata
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from wordcloud import WordCloud

import MVP_Bojado

import warnings
warnings.filterwarnings("ignore")

In [2]:
s3 = boto3.resource('s3')

# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)

ds_job_bucket = s3.Bucket('dsrawjobpostings')

dspreparedjobpostings
dsrawjobpostings
wdpreparedjobpostings
wdrawjobpostings


In [10]:
for page in ds_job_bucket.objects.pages():
    for obj in page:
        print(obj.key)

In [3]:
ds_job_bucket.download_file('dsrawjobpostings.csv', 'dsrawjobpostings.csv')

In [4]:
def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [5]:
upload_file('dsrawjobpostings.csv', 'dsrawjobpostings', 'dsrawjobpostings.csv')

True